2025/11/26
# Learn entanglement in action （quantum teleportation,superdense coding,CHSH game）

Before we begin ,I'd like to introduce a classic scene:

* Imagine two pens, a red one and a blue one ，and then I put them in a drawer.I randomly take one on a long trip. Later, when  I discover that  my pen is blue , I immediately realized the one left behind is red.

The above scenario looks very much like entanglement，but I need to clarify a key difference: entanglement is a kind of resource that is consumed upon use.
Forthermore, there is a fundamentally distinton:
* before we measure the quantum paticle, the state is random. But as a pen,even though I don't look at it, its color is already determinate the monent I take it.

## Quantum teleportation
Through quantum teleportation, we can transmit a quantum state and its correlations with other qubits to a destination. However, It's important to note that this is communication protocol, not physical transport. Unless the classical communication , we can't simply copy information due to no-cloning theorem. Instead we  need a pre-share 'e-bit'(entanglement bit) to achieve this. In this process, the origin qubit will be destroyed upon we transfer. The code and text below will demonstrate how it works.

In [ ]:
# In this case,  I implemented quantum teleportation from Alice0 to Bob.The result is a quantumcircuit.
from qiskit import QuantumCircuit , QuantumRegister, ClassicalRegister

reg_R = QuantumRegister(1, 'reg_R')  
reg_Alice = QuantumRegister(2, 'reg_Alice')  #qubit Q and A
reg_Bob = QuantumRegister(1, 'reg_Bob')
creg_Alice = ClassicalRegister(2, 'creg_Alice')

qc = QuantumCircuit(reg_R, reg_Alice, reg_Bob, creg_Alice)

qc.h(reg_R[0])
qc.cx(reg_R[0], reg_Alice[0])

# Entanglement between Alice and Bob
qc.h(reg_Alice[1])
qc.cx(reg_Alice[1], reg_Bob[0])
qc.barrier(label='pi_0')

qc.cx(reg_Alice[0], reg_Alice[1])
qc.barrier(label='pi_1')
qc.h(reg_Alice[0])
qc.barrier(label='pi_2')
qc.measure(reg_Alice[1], creg_Alice[1])
qc.measure(reg_Alice[0], creg_Alice[0])

with qc.if_test((creg_Alice, 1)):
    qc.z(reg_Bob[0])
with qc.if_test((creg_Alice, 3)):
    qc.z(reg_Bob[0])
with qc.if_test((creg_Alice, 2)):
    qc.x(reg_Bob[0])
with qc.if_test((creg_Alice, 3)):
    qc.x(reg_Bob[0])

qc.draw('mpl')

# mathematical representation:


### Explanation of Quantum teleportaton protocal ：
In the QuantumCircuit,the registers {reg_R....reg_Bob} correspond to {R,Q,A,B}.Our goal is to transmit the qubit Q to Bob. 

Initially,system can be presented by $\ket{\pi_0}$:
This is the  tensor product of The Bell state $\ket{\phi^+}_{BA}$ and  the  $Q$ entangled  with $R$ ：$$\ket{\pi_0} = \ket{\phi^+}_{BA} \otimes (\alpha\ket{0}_Q\ket{\gamma_0}_R + \beta\ket{1}_Q\ket{\gamma_1}_R)$$
We expand  the formula in the order  $(B, R, A, Q)$ :
$$\ket{\pi_0} = \frac{1}{\sqrt{2}} (\alpha\ket{0}_B\ket{\gamma_0}_R\ket{00}_{AQ} + \alpha\ket{1}_B\ket{\gamma_0}_R\ket{10}_{AQ} + \beta\ket{0}_B\ket{\gamma_1}_R\ket{01}_{AQ} + \beta\ket{1}_B\ket{\gamma_1}_R\ket{11}_{AQ})$$
Step 1: Applys a CNOT gate(control Q,target A)from Q to A.The system evolves to $\ket{\pi_1}$:
$$\ket{\pi_1} = \frac{1}{\sqrt{2}} (\alpha\ket{0}_B\ket{\gamma_0}_R\ket{00}_{AQ} + \alpha\ket{1}_B\ket{\gamma_0}_R\ket{10}_{AQ} + \beta\ket{0}_B\ket{\gamma_1}_R\ket{11}_{AQ} + \beta\ket{1}_B\ket{\gamma_1}_R\ket{01}_{AQ})$$

Step 2: Applys a H gate to Q,system evolves to $\ket{\pi_2}$:
$$\ket{\pi_2} = \frac{1}{2}(\alpha\ket{0}_B\ket{\gamma_0}_R + \beta\ket{1}_B\ket{\gamma_1}_R)\ket{00}_{AQ} + \frac{1}{2}(\alpha\ket{0}_B\ket{\gamma_0}_R - \beta\ket{1}_B\ket{\gamma_1}_R)\ket{01}_{AQ} + \frac{1}{2}(\alpha\ket{1}_B\ket{\gamma_0}_R + \beta\ket{0}_B\ket{\gamma_1}_R)\ket{10}_{AQ} + \frac{1}{2}(\alpha\ket{1}_B\ket{\gamma_0}_R - \beta\ket{0}_B\ket{\gamma_1}_R)\ket{11}_{AQ}$$

By measuring Alice's qubits,we obtain four possible results $\ket{m_A m_Q} \in \{\ket{00}, \ket{01}, \ket{10}, \ket{11}\}$ .Based on these results,Bob applys the corresponding corrections(Pauli gates).finally the system  $(B, R)$ collapses to the target state:$$\alpha\ket{0}_B\ket{\gamma_0}_R + \beta\ket{1}_B\ket{\gamma_1}_R$$
Conclusion:While R remains passive(it just 'coms along for the ride'), we have successfully transferred the state of Q to B,perserveing the entanglement with R.

In [ ]:
# I write this part to show that the final state of (R, B) is same as the initial state of (R, Q)

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import Statevector, partial_trace, state_fidelity, DensityMatrix
import numpy as np


q_Q = QuantumRegister(1, 'Q') 
q_A = QuantumRegister(1, 'A') 
q_R = QuantumRegister(1, 'R') 
q_B = QuantumRegister(1, 'B') 

# ===  set initial entanglemen parameters ===
theta = np.pi / 3

qc_target = QuantumCircuit(q_R, q_Q)
qc_target.ry(theta, q_R)
qc_target.cx(q_R, q_Q)

state_RQ_init = Statevector.from_instruction(qc_target)
display(state_RQ_init.draw('latex'))


qc = QuantumCircuit(q_B, q_R, q_A, q_Q) 

# -- initial --
qc.ry(theta, q_R) 
qc.cx(q_R, q_Q)
qc.h(q_B)
qc.cx(q_B, q_A)

# -- teleportation --
qc.cx(q_Q, q_A) 
qc.h(q_Q)       

# -- deferred measurement --
qc.cx(q_A, q_B) 
qc.cz(q_Q, q_B)


state_final_all = Statevector.from_instruction(qc)

# throw away Q and A,you will find that the state of (R, B) is same as the initial state of (R, Q)
rho_RB_final = partial_trace(state_final_all, [2, 3])
vec_RB_final = rho_RB_final.to_statevector()
display(vec_RB_final.draw('latex'))